In [15]:
import glob
from joblib import dump, load
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.lines import Line2D
import cartopy.crs as ccrs
import warnings
import matplotlib.colors as mcolors
import seaborn as sns
import numpy as np
from scipy.stats import pearsonr

## Gust processing

### Create a dataframe for gust with all campgrounds and the corresponding coordinates and nearest grid cells

In [16]:
# Define the locations and their coordinates
locations = {
    "Bern": (2595842.15, 1201548.69),
    "Brienz": (2646563.43, 1177603.88),
    "Buochs": (2674514.65, 1203568.08),
    "Interlaken": (2632995.62, 1171399.67),
    "Lugano": (2713926.15, 1094747.56),
    "Salavaux": (2569175.20, 1195846.24),
    "Sempach": (2657011.60, 1219718.50),
    "Gordevio": (2700590.96, 1119701.60),
    "Thun": (2614554.68, 1178201.33),
    "Morges": (2527116.10, 1150785.39)
}

# Open your datasets
path = '/scratch2/cwenger/ICON-CH1_monthly/ICON-CH1_gust_data/ICON-CH1-EPS_2024-08-12T12_agg.nc'
ds = xr.open_dataset(path)  # Replace with your actual dataset

# Loop through each location to find the nearest x and y and save variables
for name, (loc_x, loc_y) in locations.items():
    # Find the nearest grid cell
    nearest_x = ds.sel(x=loc_x, y=loc_y, method="nearest").x.values
    nearest_y = ds.sel(x=loc_x, y=loc_y, method="nearest").y.values

    # Save variables dynamically
    globals()[f"{name}_cell_x"] = nearest_x
    globals()[f"{name}_cell_y"] = nearest_y


locations_df = pd.DataFrame([(name, coord[0], coord[1]) for name, coord in locations.items()], columns=['name', 'x_coordinate', 'y_coordinate'])
# Add x_cell and y_cell columns to the dataframe
locations_df['x_cell_nearest'] = locations_df['name'].apply(lambda name: globals().get(f"{name}_cell_x"))
locations_df['y_cell_nearest'] = locations_df['name'].apply(lambda name: globals().get(f"{name}_cell_y"))
dump(locations_df, 'campgrounds_coordinates_gust.pkl')

['campgrounds_coordinates_gust.pkl']

### Load campground dataframe and select subset of dataset that only considers the chosen campground for gust

In [17]:
campgrounds_gust = load('campgrounds_coordinates_gust.pkl')

In [18]:
# Open your gust dataset
icon_path_gust = '/scratch2/cwenger/ICON-CH1_monthly/ICON-CH1_gust_data/ICON-CH1-EPS_2024-08-12T12_agg.nc'
ds_icon_gust = xr.open_dataset(icon_path_gust)

### Choose camping site for gust plot

In [19]:
# CHOOSE CAMPING SITE
c_name = 'Brienz'


# Define the window size (in grid points)
full_window = 35
half_window = int((full_window-1)/2)

center_x = campgrounds_gust[campgrounds_gust['name'].str.contains(c_name)]['x_cell_nearest'].values[0]
center_y = campgrounds_gust[campgrounds_gust['name'].str.contains(c_name)]['y_cell_nearest'].values[0]

# Get the indices of the nearest grid point
x_index = int(ds_icon_gust.x.to_index().get_loc(center_x))
y_index = int(ds_icon_gust.y.to_index().get_loc(center_y))

# Compute index bounds
x_start_idx = max(x_index - half_window, 0)
x_end_idx = min(x_index + half_window + 1, ds_icon_gust.sizes['x'])
y_start_idx = max(y_index - half_window, 0)
y_end_idx = min(y_index + half_window + 1, ds_icon_gust.sizes['y'])

# Extract the window for gust data
window_icon_gust = ds_icon_gust.isel(x=slice(x_start_idx, x_end_idx), y=slice(y_start_idx, y_end_idx))

#### window_icon now contains only the gust data for the chosen campground with a 35kmx35km window around it

### Set the date and the leadtime as variables

In [20]:
selected_date = '2024081212'
selected_leadtime = 0

### Print statistics for a selected date, leadtime, and member

In [7]:
print(f"\n--- Some statistics for {c_name} at {selected_date}, leadtime {selected_leadtime} ---")

# Over one specific member
for member in range(11):
    print(f"\n---  Member {member} ---")
    wind_gust = window_icon_gust.sel(lead_time=selected_leadtime, realization=member).wind_speed_of_gust
    max_wind_gust_member = wind_gust.max().values
    min_wind_gust_member = wind_gust.min().values
    mean_wind_gust_member = wind_gust.mean().values

    print("Max. wind gust:", max_wind_gust_member, "m s⁻¹")
    print("Min. wind gust:", min_wind_gust_member, "m s⁻¹")
    print("Mean wind gust:", mean_wind_gust_member, "m s⁻¹")

# Over all members
max_wind_gust = []
min_wind_gust = []
mean_wind_gust = []

for member in range(11):
    wind = window_icon_gust.sel(lead_time=selected_leadtime, realization=member).wind_speed_of_gust
    max_wind_gust.append(wind.max().values)
    min_wind_gust.append(wind.min().values)
    mean_wind_gust.append(wind.mean().values)
    
print("\n--- Over all 11 members ---")
print("Max. wind gust:", np.max(max_wind_gust), "m s⁻¹")
print("Min. wind gust:", np.min(min_wind_gust), "m s⁻¹")
print("Mean wind gust:", np.mean(mean_wind_gust), "m s⁻¹")

# Save statistics to a text file
c_name_lower = c_name[0].lower() + c_name[1:]
directory_to_save = f'/scratch2/cwenger/ICON-CH1_monthly/figures_{c_name_lower}'
with open(f'{directory_to_save}/{selected_date}_{c_name_lower}_statistics_gust_time{selected_leadtime}.txt', 'w') as f:
    f.write(f"\n--- Some statistics for {c_name} at {selected_date}, leadtime {selected_leadtime} ---\n")
    for member in range(11):
        wind_gust = window_icon_gust.sel(lead_time=selected_leadtime, realization=member).wind_speed_of_gust
        max_wind_gust_member = wind_gust.max().values
        min_wind_gust_member = wind_gust.min().values
        mean_wind_gust_member = wind_gust.mean().values
        f.write(f"\n--- Member {member} ---\n")
        f.write(f"Max. wind gust: {max_wind_gust_member} m s⁻¹\n")
        f.write(f"Min. wind gust: {min_wind_gust_member} m s⁻¹\n")
        f.write(f"Mean wind gust: {mean_wind_gust_member} m s⁻¹\n")

    f.write(f"\n--- Over all 11 members ---\n")
    f.write(f"Max. wind gust: {np.max(max_wind_gust)} m s⁻¹\n")
    f.write(f"Min. wind gust: {np.min(min_wind_gust)} m s⁻¹\n")
    f.write(f"Mean wind gust: {np.mean(mean_wind_gust)} m s⁻¹\n")



--- Some statistics for Brienz at 2024081212, leadtime 0 ---

---  Member 0 ---
Max. wind gust: 32.907223 m s⁻¹
Min. wind gust: 8.152565 m s⁻¹
Mean wind gust: 16.89212 m s⁻¹

---  Member 1 ---
Max. wind gust: 33.463184 m s⁻¹
Min. wind gust: 6.614568 m s⁻¹
Mean wind gust: 17.572178 m s⁻¹

---  Member 2 ---
Max. wind gust: 36.693356 m s⁻¹
Min. wind gust: 7.8836584 m s⁻¹
Mean wind gust: 17.49199 m s⁻¹

---  Member 3 ---
Max. wind gust: 38.77246 m s⁻¹
Min. wind gust: 5.9153585 m s⁻¹
Mean wind gust: 19.106892 m s⁻¹

---  Member 4 ---
Max. wind gust: 35.77529 m s⁻¹
Min. wind gust: 7.372965 m s⁻¹
Mean wind gust: 19.626343 m s⁻¹

---  Member 5 ---
Max. wind gust: 34.40051 m s⁻¹
Min. wind gust: 7.572669 m s⁻¹
Mean wind gust: 20.056686 m s⁻¹

---  Member 6 ---
Max. wind gust: 29.977957 m s⁻¹
Min. wind gust: 7.676999 m s⁻¹
Mean wind gust: 15.546334 m s⁻¹

---  Member 7 ---
Max. wind gust: 40.19412 m s⁻¹
Min. wind gust: 7.931038 m s⁻¹
Mean wind gust: 19.874674 m s⁻¹

---  Member 8 ---
Max. wind g

### Plot gust wind speed for the specific campground and the window around the campground

In [ ]:
warnings.filterwarnings("ignore")

# plot the actual data, for a specific date, time step, and member
for member_number in range(11): # Loop through members 0 to 10
    data_wind = window_icon_gust.sel(lead_time=selected_leadtime, realization=member_number).wind_speed_of_gust.squeeze()

    # Define the Swiss coordinate system (EPSG:2056)
    swiss_proj = ccrs.epsg(2056)

    # Compute the extent of the data
    min_x, max_x = data_wind.x.values.min(), data_wind.x.values.max()
    min_y, max_y = data_wind.y.values.min(), data_wind.y.values.max()
    extent_x = max_x - min_x
    extent_y = max_y - min_y

    # Dynamically adjust figure size to maintain a square aspect ratio
    fig_size = 13  # Base figure size
    aspect_ratio = extent_y / extent_x
    fig_width = fig_size
    fig_height = fig_size * aspect_ratio if aspect_ratio >= 1 else fig_size
    fig_width = fig_size / aspect_ratio if aspect_ratio < 1 else fig_size

    # Create the figure and axes
    fig = plt.figure(figsize=(fig_width, fig_height))
    ax = plt.axes(projection=swiss_proj)
    ax.set_aspect('1')  # Ensure square aspect ratio for the map

    # Set the extent of the map
    ax.set_extent([min_x, max_x, min_y, max_y], crs=swiss_proj)


    # Add the WMS layer for Switzerland's topography
    wms_url = 'https://wms.geo.admin.ch/?'
    layer = 'ch.swisstopo.pixelkarte-grau'
    ax.add_wms(wms_url, layer)

    # Add the WMS layer for Switzerland's borders
    border_layer = 'ch.swisstopo.swissboundaries3d-land-flaeche.fill'
    ax.add_wms(wms_url, layers=border_layer)
    canton_layer = 'ch.swisstopo.swissboundaries3d-kanton-flaeche.fill'  # Cantonal borders


    # Define colormap and normalization
    colors = [
    '#cccccc',  # 0–3
    '#66ff00',  # 3–6
    "#b3ff00",  # 6–9
    "#fffb00",  # 9–12
    '#ff9900',  # 12–15
    "#ff3c00",  # 15–18
    '#ff0000',  # 18–21
    "#ff0062",  # 21–24
    "#ff00bfdd",  # 24–27
    "#ae00ff3e",  # 27–30
    "#00000000",  # >30
    ]
    cmap = mcolors.ListedColormap(colors)
    bounds = [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30]
    norm = mcolors.BoundaryNorm(bounds, cmap.N - 1)


    # Plot the data
    im = ax.contourf(
        data_wind.x,
        data_wind.y,
        data_wind,
        levels=bounds,
        transform=swiss_proj,
        cmap=cmap, norm=norm,
        alpha=0.85, extend='max'
    )

    # add blue contour line for threshold
    threshold_line = 18

    cs = ax.contour(
        data_wind.x,
        data_wind.y,
        data_wind,
        levels=[threshold_line],
        colors='blue',
        linewidths=2.0,
        transform=swiss_proj,
        linestyles='-'
    )
        # Add legend
    legend_lines = [Line2D([0], [0], color="blue", lw=2.2)]
    legend_labels = [f'{threshold_line} m s⁻¹']
    legend = ax.legend(
        handles=legend_lines,
        labels=legend_labels,
        loc='upper left',
        bbox_to_anchor=(-0.295, 1.0),
        fontsize=16,
        framealpha=0.9,
        title='Warning Threshold',
        title_fontsize=16
    )
    legend._legend_box.align = "left"


    # Add the colorbar
    cbar = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.02, aspect=30, shrink=0.825)
    cbar.set_label('Wind Gust [m s⁻¹]', fontsize=16)
    cbar.set_ticks(bounds)
    cbar.ax.yaxis.set_major_locator(mticker.FixedLocator(bounds))
    cbar.set_ticklabels([str(b) for b in bounds])
    cbar.ax.tick_params(labelsize=16)
    fig.suptitle(f'{c_name}', fontsize=18, y=0.845, x=0.44)

    # Add dot where camping site is located
    ax.plot(campgrounds_gust[campgrounds_gust['name'].str.contains(c_name)]['x_cell_nearest'].values[0], campgrounds_gust[campgrounds_gust['name'].str.contains(c_name)]['y_cell_nearest'].values[0], 'ro', transform=swiss_proj)

    # Extract date, time, and member values
    date_val = str(selected_date)
    time_val = int(data_wind.coords['lead_time'].values)
    member_val = int(data_wind.coords['realization'].values)


    # Save figure
    directory_to_save = f'/scratch2/cwenger/ICON-CH1_monthly/figures_{c_name_lower}'
    plt.savefig(f'{directory_to_save}/{date_val}_{c_name_lower}_gust_time{time_val}_member{member_val}.png', dpi=150, bbox_inches='tight')
    


## Precipitation processing

### Create a dataframe for precipitaton with all campgrounds and the corresponding coordinates and nearest grid cells

In [8]:
# Define the locations and their coordinates
locations = {
    "Bern": (2595842.15, 1201548.69),
    "Brienz": (2646563.43, 1177603.88),
    "Buochs": (2674514.65, 1203568.08),
    "Interlaken": (2632995.62, 1171399.67),
    "Lugano": (2713926.15, 1094747.56),
    "Salavaux": (2569175.20, 1195846.24),
    "Sempach": (2657011.60, 1219718.50),
    "Gordevio": (2700590.96, 1119701.60),
    "Thun": (2614554.68, 1178201.33),
    "Morges": (2527116.10, 1150785.39)
}

# Open your datasets
path = '/scratch2/cwenger/ICON-CH1_monthly/ICON-CH1_08_2024_agg.nc'
ds = xr.open_dataset(path)  # Replace with your actual dataset

# Loop through each location to find the nearest x and y and save variables
for name, (loc_x, loc_y) in locations.items():
    # Find the nearest grid cell
    nearest_x = ds.sel(x=loc_x, y=loc_y, method="nearest").x.values
    nearest_y = ds.sel(x=loc_x, y=loc_y, method="nearest").y.values

    # Save variables dynamically
    globals()[f"{name}_cell_x"] = nearest_x
    globals()[f"{name}_cell_y"] = nearest_y


locations_df = pd.DataFrame([(name, coord[0], coord[1]) for name, coord in locations.items()], columns=['name', 'x_coordinate', 'y_coordinate'])
# Add x_cell and y_cell columns to the dataframe
locations_df['x_cell_nearest'] = locations_df['name'].apply(lambda name: globals().get(f"{name}_cell_x"))
locations_df['y_cell_nearest'] = locations_df['name'].apply(lambda name: globals().get(f"{name}_cell_y"))
dump(locations_df, 'campgrounds_coordinates_precip.pkl')

['campgrounds_coordinates_precip.pkl']

### Load campground dataframe and select subset of dataset that only considers the chosen campground for precipitation

In [9]:
campgrounds_precip = load('campgrounds_coordinates_precip.pkl')

In [10]:
# Open your precipitation dataset
icon_path_precip = '/scratch2/cwenger/ICON-CH1_monthly/ICON-CH1_08_2024_agg.nc'
ds_icon_precip = xr.open_dataset(icon_path_precip)

### Choose camping site for precipitation plot

In [11]:
# CHOOSE CAMPING SITE
c_name = 'Brienz'


# Define the window size (in grid points)
full_window = 35
half_window = int((full_window-1)/2)

center_x = campgrounds_precip[campgrounds_precip['name'].str.contains(c_name)]['x_cell_nearest'].values[0]
center_y = campgrounds_precip[campgrounds_precip['name'].str.contains(c_name)]['y_cell_nearest'].values[0]

# Get the indices of the nearest grid point
x_index = int(ds_icon_precip.x.to_index().get_loc(center_x))
y_index = int(ds_icon_precip.y.to_index().get_loc(center_y))

# Compute index bounds
x_start_idx = max(x_index - half_window, 0)
x_end_idx = min(x_index + half_window + 1, ds_icon_precip.sizes['x'])
y_start_idx = max(y_index - half_window, 0)
y_end_idx = min(y_index + half_window + 1, ds_icon_precip.sizes['y'])

# Extract the window for precipitation data
window_icon_precip = ds_icon_precip.isel(x=slice(x_start_idx, x_end_idx), y=slice(y_start_idx, y_end_idx))

#### window_icon now contains only the precipitation data for the chosen campground with a 35kmx35km window around it

### Plot the precipitation for the specific campground and the window around the campground

In [ ]:
warnings.filterwarnings("ignore")

# Plot the actual data, for a specific date, time step, and member
for member_number in range(11): # Loop through members 0 to 10
    data_prec = window_icon_precip.sel(date=selected_date, time=selected_leadtime, members=member_number).tot_prec_1h

    # Define the Swiss coordinate system (EPSG:2056)
    swiss_proj = ccrs.epsg(2056)

    # Compute the extent of the data
    min_x, max_x = data_prec.x.values.min(), data_prec.x.values.max()
    min_y, max_y = data_prec.y.values.min(), data_prec.y.values.max()
    extent_x = max_x - min_x
    extent_y = max_y - min_y

    # Dynamically adjust figure size to maintain a square aspect ratio
    fig_size = 13  # Base figure size
    aspect_ratio = extent_y / extent_x
    fig_width = fig_size
    fig_height = fig_size * aspect_ratio if aspect_ratio >= 1 else fig_size
    fig_width = fig_size / aspect_ratio if aspect_ratio < 1 else fig_size

    # Create the figure and axes
    fig = plt.figure(figsize=(fig_width, fig_height))
    ax = plt.axes(projection=swiss_proj)
    ax.set_aspect('1')  # Ensure square aspect ratio for the map

    # Set the extent of the map
    ax.set_extent([min_x, max_x, min_y, max_y], crs=swiss_proj)


    # Add the WMS layer for Switzerland's topography
    wms_url = 'https://wms.geo.admin.ch/?'
    layer = 'ch.swisstopo.pixelkarte-grau'
    ax.add_wms(wms_url, layer)

    # Add the WMS layer for Switzerland's borders
    border_layer = 'ch.swisstopo.swissboundaries3d-land-flaeche.fill'
    ax.add_wms(wms_url, layers=border_layer)
    canton_layer = 'ch.swisstopo.swissboundaries3d-kanton-flaeche.fill'  # Cantonal borders


    # Define colormap and normalization
    colors = ['#eafac3', '#b3f8c0', '#58c1a0', '#32a1a0', '#0f79b1', '#06488e', '#1f0d64', '#6a2c5a']
    cmap = mcolors.ListedColormap(colors)
    bounds = [1, 2, 4, 6, 10, 20, 40, 60]
    norm = mcolors.BoundaryNorm(bounds, cmap.N - 1)


    # Plot the data
    im = ax.contourf(
        data_prec.x,
        data_prec.y,
        data_prec,
        levels=bounds,
        vmin=1, vmax=60,
        transform=swiss_proj,
        cmap=cmap, norm=norm,
        alpha=0.85, extend='max'
    )

    # add red contour line for threshold
    threshold_line = 13.6

    cs = ax.contour(
        data_prec.x,
        data_prec.y,
        data_prec,
        levels=[threshold_line],
        colors='red',
        linewidths=2.0,
        transform=swiss_proj,
        linestyles='-'
    )
        # Add legend
    legend_lines = [Line2D([0], [0], color="red", lw=2.2)]
    legend_labels = [f'{threshold_line} mm h⁻¹']
    legend = ax.legend(
        handles=legend_lines,
        labels=legend_labels,
        loc='upper left',
        bbox_to_anchor=(-0.295, 1.0),
        fontsize=16,
        framealpha=0.9,
        title='Warning Threshold',
        title_fontsize=16
    )
    legend._legend_box.align = "left"


    # Add the colorbar
    cbar = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.02, aspect=30,shrink=0.825)
    cbar.set_label('Precipitation [mm h⁻¹]', fontsize=16)
    cbar.ax.tick_params(labelsize=16)
    fig.suptitle(f'{c_name}', fontsize=18, y=0.845, x=0.44)

    # Add dot where camping site is located
    ax.plot(campgrounds_precip[campgrounds_precip['name'].str.contains(c_name)]['x_cell_nearest'].values[0], campgrounds_precip[campgrounds_precip['name'].str.contains(c_name)]['y_cell_nearest'].values[0], 'ro', transform=swiss_proj)

    # Extract date, time, and member values
    date_val = str(data_prec.coords['date'].values)
    time_val = int(data_prec.coords['time'].values)
    member_val = int(data_prec.coords['members'].values)
    c_name_lower = c_name[0].lower() + c_name[1:]

    # Save figure
    directory_to_save = f'/scratch2/cwenger/ICON-CH1_monthly/figures_{c_name_lower}'
    plt.savefig(f'{directory_to_save}/{date_val}_{c_name_lower}_precipitation_time{time_val}_member{member_val}.png', dpi=150, bbox_inches='tight')



### Place the wind contours over the precipitation plot for the specific campground and the window around the campground

In [ ]:
from matplotlib.lines import Line2D

warnings.filterwarnings("ignore")

for member_number in range(11):  # Loop through members 0 to 10
    data_prec = window_icon_precip.sel(date=selected_date, time=selected_leadtime, members=member_number).tot_prec_1h
    data_wind = window_icon_gust.sel(lead_time=selected_leadtime, realization=member_number).wind_speed_of_gust.squeeze()

    swiss_proj = ccrs.epsg(2056)

    min_x, max_x = data_prec.x.values.min(), data_prec.x.values.max()
    min_y, max_y = data_prec.y.values.min(), data_prec.y.values.max()
    extent_x = max_x - min_x
    extent_y = max_y - min_y

    fig_size = 13
    aspect_ratio = extent_y / extent_x
    fig_width = fig_size
    fig_height = fig_size * aspect_ratio if aspect_ratio >= 1 else fig_size
    fig_width = fig_size / aspect_ratio if aspect_ratio < 1 else fig_size

    fig = plt.figure(figsize=(fig_width, fig_height))
    ax = plt.axes(projection=swiss_proj)
    ax.set_aspect('1')
    ax.set_extent([min_x, max_x, min_y, max_y], crs=swiss_proj)

    # Add the WMS layer for Switzerland's topography
    wms_url = 'https://wms.geo.admin.ch/?'
    layer = 'ch.swisstopo.pixelkarte-grau'
    ax.add_wms(wms_url, layer)

    # Add the WMS layer for Switzerland's borders
    border_layer = 'ch.swisstopo.swissboundaries3d-land-flaeche.fill'
    ax.add_wms(wms_url, layers=border_layer)
    canton_layer = 'ch.swisstopo.swissboundaries3d-kanton-flaeche.fill'  # Cantonal borders

    prec_colors = ['#eafac3', '#b3f8c0', '#58c1a0', '#32a1a0', '#0f79b1', '#06488e', '#1f0d64', '#6a2c5a']
    prec_cmap = mcolors.ListedColormap(prec_colors)
    prec_bounds = [1, 2, 4, 6, 10, 20, 40, 60]
    prec_norm = mcolors.BoundaryNorm(prec_bounds, prec_cmap.N - 1)

    im = ax.contourf(
        data_prec.x,
        data_prec.y,
        data_prec,
        levels=prec_bounds,
        vmin=1, vmax=60,
        transform=swiss_proj,
        cmap=prec_cmap, norm=prec_norm,
        alpha=0.85, extend='max'
    )

    # Wind contour thresholds and colors
    bounds = [3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33]
    wind_colors = [
        '#cccccc',  # 0–3
        '#66ff00',  # 3–6
        "#b3ff00",  # 6–9
        "#fffb00",  # 9–12
        '#ff9900',  # 12–15
        "#ff3c00",  # 15–18
        '#ff0000',  # 18–21
        "#ff0062",  # 21–24
        "#ff00bfdd",  # 24–27
        "#ae00ff3e",  # 27–30
        "#000000",  # >30
    ]

    wind_contour_levels = bounds

    wind_contour = ax.contour(
        data_wind.x,
        data_wind.y,
        data_wind,
        levels=wind_contour_levels,
        colors=wind_colors,
        linewidths=2.2,
        transform=swiss_proj
    )

    ax.clabel(wind_contour, inline=True, fontsize=18, fmt='%d')

    cbar = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.02, aspect=30, shrink=0.825)
    cbar.set_label('Precipitation [mm h⁻¹]', fontsize=16)
    cbar.ax.tick_params(labelsize=16)
    fig.suptitle(f'{c_name}', fontsize=18, y=0.845, x=0.44)

    # Add legend for wind speed contours
    wind_legend_lines = [Line2D([0], [0], color=color, lw=2.2) for color in wind_colors]
    wind_legend_labels = [f'{b} m s⁻¹ ' for b in bounds]
    legend = ax.legend(
        handles=wind_legend_lines,
        labels=wind_legend_labels,
        loc='upper left',
        bbox_to_anchor=(-0.269, 1.0),
        fontsize=16,
        framealpha=0.9,
        title='Wind Gust\nContours [m s⁻¹]',
        title_fontsize=16
    )
    legend._legend_box.align = "left"

    ax.plot(
        campgrounds_precip[campgrounds_precip['name'].str.contains(c_name)]['x_cell_nearest'].values[0],
        campgrounds_precip[campgrounds_precip['name'].str.contains(c_name)]['y_cell_nearest'].values[0],
        'ro', transform=swiss_proj
    )

    date_val = str(data_prec.coords['date'].values)
    time_val = int(data_prec.coords['time'].values)
    member_val = int(data_prec.coords['members'].values)
    c_name_lower = c_name[0].lower() + c_name[1:]

    # Save figure
    directory_to_save = f'/scratch2/cwenger/ICON-CH1_monthly/figures_{c_name_lower}'
    plt.savefig(f'{directory_to_save}/{date_val}_{c_name_lower}_gust_contours_precipitation_time{time_val}_member{member_val}.png', dpi=150, bbox_inches='tight')


### Wind contours plotted over precipitation only for above 18 m s⁻¹ wind speed

In [ ]:
warnings.filterwarnings("ignore")

for member_number in range(11):  # Loop through members 0 to 10
    data_prec = window_icon_precip.sel(date=selected_date, time=selected_leadtime, members=member_number).tot_prec_1h
    data_wind = window_icon_gust.sel(lead_time=selected_leadtime, realization=member_number).wind_speed_of_gust.squeeze()

    swiss_proj = ccrs.epsg(2056)

    min_x, max_x = data_prec.x.values.min(), data_prec.x.values.max()
    min_y, max_y = data_prec.y.values.min(), data_prec.y.values.max()
    extent_x = max_x - min_x
    extent_y = max_y - min_y

    fig_size = 13
    aspect_ratio = extent_y / extent_x
    fig_width = fig_size
    fig_height = fig_size * aspect_ratio if aspect_ratio >= 1 else fig_size
    fig_width = fig_size / aspect_ratio if aspect_ratio < 1 else fig_size

    fig = plt.figure(figsize=(fig_width, fig_height))
    ax = plt.axes(projection=swiss_proj)
    ax.set_aspect('1')
    ax.set_extent([min_x, max_x, min_y, max_y], crs=swiss_proj)

    # Add the WMS layer for Switzerland's topography
    wms_url = 'https://wms.geo.admin.ch/?'
    layer = 'ch.swisstopo.pixelkarte-grau'
    ax.add_wms(wms_url, layer)

    # Add the WMS layer for Switzerland's borders
    border_layer = 'ch.swisstopo.swissboundaries3d-land-flaeche.fill'
    ax.add_wms(wms_url, layers=border_layer)
    canton_layer = 'ch.swisstopo.swissboundaries3d-kanton-flaeche.fill'  # Cantonal borders

    prec_colors = ['#eafac3', '#b3f8c0', '#58c1a0', '#32a1a0', '#0f79b1', '#06488e', '#1f0d64', '#6a2c5a']
    prec_cmap = mcolors.ListedColormap(prec_colors)
    prec_bounds = [1, 2, 4, 6, 10, 20, 40, 60]
    prec_norm = mcolors.BoundaryNorm(prec_bounds, prec_cmap.N - 1)

    im = ax.contourf(
        data_prec.x,
        data_prec.y,
        data_prec,
        levels=prec_bounds,
        vmin=1, vmax=60,
        transform=swiss_proj,
        cmap=prec_cmap, norm=prec_norm,
        alpha=0.85, extend='max'
    )

    # Wind contour thresholds and colors
    bounds = [18, 21, 24, 27, 30, 33]
    wind_colors = [
        '#ff0000',  # 18–21
        "#ff0062",  # 21–24
        "#ff00bfdd",  # 24–27
        "#ae00ff3e",  # 27–30
        "#000000",  # >30
    ][:len(bounds)]


    # Only show contours above 18 m s⁻¹
    wind_contour_levels = [b for b in bounds if b >= 18]

    wind_contour = ax.contour(
        data_wind.x,
        data_wind.y,
        data_wind,
        levels=wind_contour_levels,
        colors=wind_colors[-len(wind_contour_levels):],
        linewidths=2.2,
        transform=swiss_proj
    )

    ax.clabel(wind_contour, inline=True, fontsize=18, fmt='%d')

    cbar = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.02, aspect=30, shrink=0.825)
    cbar.set_label('Precipitation [mm h⁻¹]', fontsize=16)
    cbar.ax.tick_params(labelsize=16)
    fig.suptitle(f'{c_name}', fontsize=18, y=0.845, x=0.44)

    # Add legend for wind speed contours
    wind_legend_lines = [Line2D([0], [0], color=color, lw=2.2) for color in wind_colors]
    wind_legend_labels = [f'{b} m s⁻¹' for b in bounds]
    legend = ax.legend(
        handles=wind_legend_lines,
        labels=wind_legend_labels,
        loc='upper left',
        bbox_to_anchor=(-0.269, 1.0),
        fontsize=16,
        framealpha=0.9,
        title='Wind Gust\nContours [m s⁻¹]',
        title_fontsize=16
    )
    legend._legend_box.align = "left"

    ax.plot(
        campgrounds_precip[campgrounds_precip['name'].str.contains(c_name)]['x_cell_nearest'].values[0],
        campgrounds_precip[campgrounds_precip['name'].str.contains(c_name)]['y_cell_nearest'].values[0],
        'ro', transform=swiss_proj
    )

    date_val = str(data_prec.coords['date'].values)
    time_val = int(data_prec.coords['time'].values)
    member_val = int(data_prec.coords['members'].values)
    c_name_lower = c_name[0].lower() + c_name[1:]

    # Save figure
    directory_to_save = f'/scratch2/cwenger/ICON-CH1_monthly/figures_{c_name_lower}'
    plt.savefig(f'{directory_to_save}/{date_val}_{c_name_lower}_gust_contours_above18_precipitation_time{time_val}_member{member_val}.png', dpi=150, bbox_inches='tight')


### Wind contours plotted for members exceeding threshold level 18 m s⁻¹ and area threshold 5% (62 km²)

In [ ]:
from matplotlib.patches import Patch

warnings.filterwarnings("ignore")

data_wind = window_icon_gust.sel(
        lead_time=selected_leadtime,
        realization=member_number
    ).wind_speed_of_gust.squeeze()

# Define the Swiss coordinate system (EPSG:2056)
swiss_proj = ccrs.epsg(2056)

# Compute the extent of the data
min_x, max_x = data_wind.x.values.min(), data_wind.x.values.max()
min_y, max_y = data_wind.y.values.min(), data_wind.y.values.max()
extent_x = max_x - min_x
extent_y = max_y - min_y

# Dynamically adjust figure size to maintain a square aspect ratio
fig_size = 13  # Base figure size
aspect_ratio = extent_y / extent_x
fig_width = fig_size
fig_height = fig_size * aspect_ratio if aspect_ratio >= 1 else fig_size
fig_width = fig_size / aspect_ratio if aspect_ratio < 1 else fig_size

# Create the figure and axes
fig = plt.figure(figsize=(fig_width, fig_height))
ax = plt.axes(projection=swiss_proj)
ax.set_aspect('1')  # Ensure square aspect ratio for the map

# Set the extent of the map
ax.set_extent([min_x, max_x, min_y, max_y], crs=swiss_proj)


# Add the WMS layer for Switzerland's topography
wms_url = 'https://wms.geo.admin.ch/?'
layer = 'ch.swisstopo.pixelkarte-grau'
ax.add_wms(wms_url, layer)

# Add the WMS layer for Switzerland's borders
border_layer = 'ch.swisstopo.swissboundaries3d-land-flaeche.fill'
ax.add_wms(wms_url, layers=border_layer)
canton_layer = 'ch.swisstopo.swissboundaries3d-kanton-flaeche.fill'  # Cantonal borders

# Define colormap and normalization
colors = [
    "#1f77b4", "#fd6500", "#2ca02c", "#d62728",
    "#9467bd", "#8c564b", "#e377c2", "#4c136d",
    "#bcbd22", "#17becf", "#f1a707"
]
cmap = mcolors.ListedColormap(colors)

threshold = 18

# Plot the 18 m s⁻¹ contour for members exceeding area threshold of 62 km²
legend_lines, legend_labels = [], []

for member_number in range(11):
    data_wind = window_icon_gust.sel(
        lead_time=selected_leadtime,
        realization=member_number
    ).wind_speed_of_gust.squeeze()

    # Count grid cells above threshold for this member
    exceed_count = int(((data_wind > threshold).sum(dim=["y", "x"])).item())

    # Require both threshold reached and area > 62 km²
    if exceed_count > 62:
        cs = ax.contour(
            data_wind.x, data_wind.y, data_wind,
            levels=[threshold],
            colors=[colors[member_number]],
            linewidths=2.0,
            linestyles="-",
            transform=swiss_proj
        )
        legend_lines.append(Line2D([0], [0], color=colors[member_number], lw=2.2))
        legend_labels.append(f"Member {member_number}")

# Compute per-grid-cell count of members exceeding threshold
exceed_per_cell = (window_icon_gust.sel(lead_time=selected_leadtime).wind_speed_of_gust > threshold).sum(dim="realization")

# Mask for cells where 8 or more members exceed threshold
mask_8plus = exceed_per_cell >= 8

# Overlay filled red area where 8 or more members exceed threshold
if np.any(mask_8plus.values):
    # Prepare a 2D Z array for plotting (contourf requires 2D)
    Z = np.squeeze(mask_8plus.astype(int).values)
    if Z.ndim != 2:
        # Try to reduce extra singleton dims, otherwise pick the first slice
        Z = np.squeeze(Z)
        if Z.ndim != 2:
            Z = Z.reshape(Z.shape[-2], Z.shape[-1])

    # Build 2D coordinate grids from 1D coords
    x_vals = exceed_per_cell.x.values
    y_vals = exceed_per_cell.y.values
    X2, Y2 = np.meshgrid(x_vals, y_vals)

    ax.contourf(
        X2, Y2, Z,
        levels=[0.5, 1.5],
        colors=["orangered"],
        alpha=0.85,
        transform=swiss_proj,
        zorder=4
    )

# Add legend (only for members exceeding thresholds)
if legend_lines:
    legend = ax.legend(
        handles=legend_lines,
        labels=legend_labels,
        loc="upper left",
        bbox_to_anchor=(-0.262, 1.0),
        fontsize=16,
        framealpha=0.9,
        title='Wind Gust Contours\nfor 18 m s⁻¹',
        title_fontsize=16
    )
    legend._legend_box.align = "left"

    # Add a second legend box below the Wind Gust Contours box for the mask_8plus
    mask_patch = Patch(facecolor="orangered", edgecolor="darkred", label=f"Warning Threshold 73%\nExceeded", alpha=0.85)
    ax.add_artist(legend)
    legend2 = ax.legend(
        handles=[mask_patch],
        loc="upper left",
        bbox_to_anchor=(-0.366, 0.55),
        fontsize=16,
        framealpha=0.9,
    )
    legend2._legend_box.align = "left"

# Title
fig.suptitle(f"{c_name}", fontsize=18, y=0.927, x=0.47)
ax.set_title(f'Members Exceeding Gust Threshold of {threshold} m s⁻¹ and Area Threshold of 62 km²', fontsize=16, pad=6)

# Add dot where camping site is located
ax.plot(campgrounds_gust[campgrounds_gust['name'].str.contains(c_name)]['x_cell_nearest'].values[0], campgrounds_gust[campgrounds_gust['name'].str.contains(c_name)]['y_cell_nearest'].values[0], 'ro', transform=swiss_proj)

# Extract date, time, and member values
date_val = str(selected_date)
time_val = int(data_wind.coords['lead_time'].values)

# Save figure
directory_to_save = f"/scratch2/cwenger/ICON-CH1_monthly/figures_{c_name_lower}"
plt.savefig(f"{directory_to_save}/{date_val}_{c_name_lower}_gust_members_exceeding_thresholds_time{time_val}.png",
            dpi=150, bbox_inches="tight")


### Precipitation contours plotted for members exceeding threshold level 2 and area threshold 5% (62 km²)

### Create a scatterplot for a first visualization of possible correlations between wind and precipitation

### Print out the important statistical numbers